In [1]:
## Libraries ##

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas
import scipy.optimize as sci
from scipy.stats import linregress
import scipy.integrate as intz
from scipy import signal
import matplotlib.colors as mcolors
import matplotlib.lines as mlines
from scipy.integrate import trapz, cumtrapz

print("done")

done


/tmp/ipykernel_14595/3887738247.py:20: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


In [2]:
experiment2 = 'abrupt-2xCO2'
experiment4 = 'abrupt-4xCO2'

experiment_list = [experiment2, experiment4]

model_list1 = ['CanESM5', 'CESM2', 'MIROC6'] # models with rtmt

model_list2 = ['CNRM-CM6-1', 'GISS-E2-1-G', 'GISS-E2-1-H', 'IPSL-CM6A-LR', 'MRI-ESM2-0'] # models without rtmt

model_list  = model_list1 + model_list2 # total model list

In [13]:
heat_storage = {}
levels = {} # vertical coord
lats = {}
percents = ['50', '55', '80']

for exp in experiment_list:
    heat_storage[exp] = {}

    for model in model_list:
        heat_storage[exp][model] = {}
        
        try:
            levels[model] = np.loadtxt('%s_levels.csv' % model, delimiter=",", dtype=float)
            
        except FileNotFoundError: # data is not available for this model
                continue
                
        try:
            lats[model] = np.loadtxt('%s_latitude.csv' % model, delimiter=",", dtype=float)
            
        except FileNotFoundError: # data is not available for this model
                continue
        
        try: # reshape into 2D array; either has 150 or 151 years
            heat_storage[exp][model] = np.reshape(np.loadtxt('heat_storage_%s_%s.csv' % (model, exp), delimiter=",", dtype=float), (150, len(levels[model])-1, len(lats[model])))
            
        except ValueError:
            heat_storage[exp][model] = np.reshape(np.loadtxt('heat_storage_%s_%s.csv' % (model, exp), delimiter=",", dtype=float), (151, len(levels[model])-1, len(lats[model])))
           
        except FileNotFoundError: # data is not available for this model
            print('No Heat Storage %s Data Found for %s' % (exp, model))
            continue

In [14]:
def find_percent_hc(heat_content_array, level_values, lat_values, runtime, percent):
    # finds subgrid depth at given percent
    # heat_content_array should be time vs depth
    # level_values should be 1D array of depths
    # runtime (int) should be runtime available for this model and simulation
    # percent (float)
    
    percent_levels = np.empty(180)

    
    for i, lat in enumerate(lat_values):
        hc_i = 0
        hc_j = 0
        
        
        if np.isnan(np.nanmean(heat_content_array)):
            percent_levels[i] = np.nan
            #print('all nan at latitude %d' % i)
            
        for j, L in enumerate(level_values):
                
            hc = heat_content_array[j, i] 
            #print(i, j, hc)
            
            if np.isnan(hc):
                #print('single nan value at %d,%d' % (i, j))
                continue
            
            if hc < percent:
                hc_j = hc
                continue
                
            elif hc >= percent:
                L_i = L
                L_j = level_values[j-1]
                hc_i = hc
                
                m, b, r, p, std = linregress([L_j, L_i], [hc_j, hc_i])

                L_ij  = (percent - b)/m
                #print(L_ij, i, j)
                
                percent_levels[i] = L_ij
                #print(percent_levels)
                break

                
    print(np.shape(percent_levels))  
    
    return percent_levels # returns level that "percent" occurs at for every time

In [16]:
## find depth at which ocean contains 80% of total heat content
## IPSL should only be missing 1999 beyond. Missing 2 files. Fix like CanESM5, might need to sortby earlier

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

levels_55p_4x = {}
levels_55p_2x = {}

for model in model_list:
    if model == 'MIROC6':
        print('done', model)
        continue
        
    print(model)
        
    levs = levels[model][1:]
    
    time4 = 150 #np.shape(data4x_GL[model])[0]
    
    if model == 'IPSL-CM6A-LR':
        time2 = 130
    else:
        time2 = 150 #np.shape(data2x_GL[model])[0]
    
    levels_55p_4x[model] = {}
    levels_55p_2x[model] = {}
    
    array1 = find_percent_hc(np.nanmean(heat_storage['abrupt-4xCO2'][model][21:150, :, :], axis=0), levs, np.arange(180), time4, 0.55)
    array2 = find_percent_hc(np.nanmean(heat_storage['abrupt-2xCO2'][model][21:150, :, :], axis=0), levs, np.arange(180), time2, 0.55)

    levels_55p_4x[model] = array1
    levels_55p_2x[model] = array2
    
    #array1.tofile('depth_55_%s_%s.csv' % (model, 'abrupt-4xCO2'), sep=',')
    #array2.tofile('depth_55_%s_%s.csv' % (model, 'abrupt-2xCO2'), sep=',')
    
    print('done', model)

CanESM5


/tmp/ipykernel_14595/1767841.py:30: RuntimeWarning: Mean of empty slice
  array1 = find_percent_hc(np.nanmean(heat_storage['abrupt-4xCO2'][model][21:150, :, :], axis=0), levs, np.arange(180), time4, 0.55)


(180,)
(180,)
done CanESM5
CESM2
(180,)


/tmp/ipykernel_14595/1767841.py:31: RuntimeWarning: Mean of empty slice
  array2 = find_percent_hc(np.nanmean(heat_storage['abrupt-2xCO2'][model][21:150, :, :], axis=0), levs, np.arange(180), time2, 0.55)


(180,)
done CESM2
done MIROC6
CNRM-CM6-1
(180,)
(180,)
done CNRM-CM6-1
GISS-E2-1-G
(180,)
(180,)
done GISS-E2-1-G
GISS-E2-1-H
(180,)
(180,)
done GISS-E2-1-H
IPSL-CM6A-LR
(180,)
(180,)
done IPSL-CM6A-LR
MRI-ESM2-0
(180,)
(180,)
done MRI-ESM2-0
